In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import shared
import os
from scipy import stats

os.environ['RACE_TYPE'] = "ju"
os.environ['FORECAST_YEAR'] = "2024"
race_type = shared.race_type()
#year = shared.forecast_year()
import time
startTime = time.time()
sns.set(rc={"figure.figsize":(16, 9)}) 

In [ ]:
# To get all years use next year
runs = pd.read_csv(f'data/runs_{shared.race_id_str()}.tsv', delimiter='\t')
#runs = runs.query("num_runs >= 1")
#runs = runs["pace"].notna()
runs = runs.dropna(subset=['pace'])
runs["log_pace"] = np.log(runs["pace"])
runs["very_slow"] = runs["pace"] > 30
#runs["pace"] = runs["pace"].clip(upper=30)
runs["num_runs"] = runs["num_runs"].clip(upper=14)
runs

In [ ]:
runs["pace"].describe()

In [ ]:
runs["very_slow"].sum()

In [ ]:
1/runs["very_slow"].mean()

In [ ]:
runs.sort_values(["pace"]).tail(20)

In [ ]:
all_years = runs.year.unique()
all_years

In [ ]:
def satterthwaite_df(column):
    """Calculate the Satterthwaite degrees of freedom for a pandas column."""
    s = column.std()  # sample standard deviation
    n = len(column)  # sample size
    
    return (s**2 / n)**2 / (s**4 / ((n-1) * n**2))



def _student_t_params(column): 
    # Calculate the sample mean and sample variance
    xbar = column.mean()
    std = column.std()

    # Define the log-likelihood function
    def log_likelihood(nu):
        return sum(stats.t.logpdf(column, df=nu, loc=xbar, scale=std))

    # Find the maximum likelihood estimate of the degrees of freedom
    nu_hat = pd.Series(range(1, 500)).apply(log_likelihood).idxmax() + 1

    satterthwaite = satterthwaite_df(column)


    return xbar, std, nu_hat, satterthwaite


for year in all_years:
    #print(year)
    year_runs = runs[runs["year"] == year]
    #xbar, std, nu_hat, satterthwaite = _student_t_params(year_runs["log_pace"])
    # Print the results
    #print(f'{year} Sample mean: {xbar:.2f}, Sample std: {std:.2f}, Degrees of freedom: {nu_hat}, satterthwaite: {satterthwaite:.1f}, min: {year_runs["pace"].min():.1f}, max: {year_runs["pace"].max():.1f}')
    # Fit a t-distribution to your data
    degrees_of_freedom, location, scale = stats.t.fit(year_runs["log_pace"])
    print(f'{year} St fit medi: {location:.2f}, Sample std: {scale:.2f}, degrees_of_freedom: {degrees_of_freedom:.1f}, min: {year_runs["pace"].min():.1f}, max: {year_runs["pace"].max():.1f}')

    

In [ ]:
degrees_of_freedom, location, scale = stats.t.fit(runs["log_pace"])
# Simulate the same amount of random data with the estimated parameters
simulated_data = stats.t.rvs(degrees_of_freedom, location, scale, size=len(runs["log_pace"]))

print(f'ALL ST fit medi: {location:.2f}, Sample std: {scale:.2f}, degrees_of_freedom: {degrees_of_freedom:.1f}, min: {sub_runs["pace"].min():.1f}, max: {sub_runs["pace"].max():.1f}')
fig, ax = plt.subplots()
sns.histplot(runs["log_pace"], bins=100, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
sns.histplot(simulated_data, bins=100, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)


In [ ]:
degrees_of_freedom, location, scale = stats.t.fit(runs["log_pace"])
# Simulate the same amount of random data with the estimated parameters
simulated_data_2 = stats.t.rvs(degrees_of_freedom, location, scale, size=len(runs["log_pace"]))

# Assuming runs is a dictionary or a pandas DataFrame with a "log_pace" key/column
min_value = np.min(runs["log_pace"]) + 0.1

# Filter values in simulated_data that are below the minimum of runs["log_pace"]
below_min_values = simulated_data_2[simulated_data_2 < min_value]

# Resample these values
resampled_values = np.random.choice(simulated_data_2, size=len(below_min_values), replace=True)

# Replace the below-minimum values in simulated_data with the resampled values
simulated_data_2[simulated_data_2 < min_value] = resampled_values

fig, ax = plt.subplots()
sns.histplot(runs["log_pace"], bins=100, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
sns.histplot(simulated_data_2, bins=100, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)


In [ ]:
degrees_of_freedom, location, scale = stats.t.fit(runs["log_pace"])
simulated_data_3 = stats.t.rvs(degrees_of_freedom, location, scale, size=len(runs["log_pace"]))
# Define the threshold for the tail
threshold = np.percentile(simulated_data_3, 85)

# Fit the Pareto distribution to the upper tail
shape_param, loc, scale_param = stats.pareto.fit(simulated_data_3[simulated_data_3 > threshold])

num_tail_values = sum(simulated_data_3 > threshold)
new_tail_data = stats.pareto.rvs(shape_param, loc, scale_param, size=num_tail_values)

simulated_data_3[simulated_data_3 > threshold] = new_tail_data



#shape_param, loc, scale_param = stats.pareto.fit(runs["log_pace"])
# Simulate the same amount of random data with the estimated parameters
#pareto_simulated_data = stats.pareto.rvs(shape_param, loc, scale_param, size=len(runs["log_pace"]))

#print(f'ALL ST fit medi: {location:.2f}, Sample std: {scale:.2f}, degrees_of_freedom: {degrees_of_freedom:.1f}, min: {sub_runs["pace"].min():.1f}, max: {sub_runs["pace"].max():.1f}')
fig, ax = plt.subplots()
sns.histplot(runs["log_pace"], bins=100, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
sns.histplot(simulated_data_3, bins=100, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)


In [ ]:
def simulate_weighted_t(data, weight_power=2):
    # Fit data to Student's t distribution
    df, loc, scale = stats.t.fit(data)
    
    # Simulate data from the estimated parameters
    simulated_data = stats.t.rvs(df, loc, scale, size=len(data))
    
    # Assign weights to simulated data
    # Here, we're raising each sample to a power to emphasize the right tail
    # The weight_power determines how much emphasis to put on the right tail
    weights = simulated_data ** weight_power
    
    # Normalize weights to make them sum to 1
    weights /= weights.sum()
    
    # Resample from the simulated data using the weights
    weighted_samples = np.random.choice(simulated_data, size=len(data), p=weights)
    
    return weighted_samples

weighted_simulated = simulate_weighted_t(runs["log_pace"], weight_power=1.7)


fig, ax = plt.subplots()
sns.histplot(runs["log_pace"], bins=30, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
sns.histplot(weighted_simulated, bins=30, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)


In [ ]:
from scipy.stats import gaussian_kde

def get_kde_samples(data, n_samples=None):
    """Get samples from KDE."""
    if n_samples is None:
        n_samples = len(data)
        
    kde = gaussian_kde(data)
    print(kde)
    # Setting bounds for the sampled data
    min_val, max_val = min(data), max(data)
    simulated_data = kde.resample(n_samples).flatten()
    
    # If the sampled data goes beyond original bounds, we resample those points
    while any(simulated_data < min_val) or any(simulated_data > max_val):
        out_of_bounds = (simulated_data < min_val) | (simulated_data > max_val)
        new_samples = kde.resample(out_of_bounds.sum()).flatten()
        simulated_data[out_of_bounds] = new_samples

    return simulated_data


simulated_data = get_kde_samples(runs["log_pace"])

fig, ax = plt.subplots()
sns.histplot(runs["log_pace"], bins=30, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
sns.histplot(simulated_data, bins=30, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)


In [ ]:
def fit_and_simulate(data, dist_name):
    if dist_name == 'gamma':
        params = stats.gamma.fit(data)
        simulated_data = stats.gamma.rvs(*params, size=len(data))
    elif dist_name == 'weibull':
        params = stats.weibull_min.fit(data)  # Note: we're using weibull_min which represents the common 3-parameter Weibull
        simulated_data = stats.weibull_min.rvs(*params, size=len(data))
    else:
        raise ValueError(f"Unknown distribution: {dist_name}")
    
    return simulated_data, params

def plot_data_and_simulated(data, simulated, dist_name):
    #plt.figure(figsize=(15, 6))
    
    sns.histplot(data, kde=True, color="blue", label="Original Data", alpha=0.5)
    sns.histplot(weibull_simulated, kde=True, color="red", label=f"{dist_name} Simulated", alpha=0.5)
    
    plt.title(f'Original vs {dist_name} Simulated Data')
    plt.legend()
    plt.show()

# Sample usage:
data = runs["log_pace"]  # Replace with your data

gamma_simulated, gamma_params = fit_and_simulate(data, 'gamma')
weibull_simulated, weibull_params = fit_and_simulate(data, 'weibull')

plot_data_and_simulated(data, gamma_simulated, "Gamma")
plot_data_and_simulated(data, weibull_simulated, "weibull")


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
all_params = []
for num_runs, sub_runs in runs.groupby("num_runs"):
    #display(sub_runs)
    #num_runs = sub_runs["num_runs"].unique()
    #xbar, std, nu_hat, satterthwaite = _student_t_params(sub_runs["log_pace"])
    # Print the results
    #print(f'{num_runs} Sample mean: {xbar:.2f}, Sample std: {std:.2f}, Degrees of freedom: {nu_hat}, min: {sub_runs["pace"].min():.1f}, max: {sub_runs["pace"].max():.1f}')
    degrees_of_freedom, location, scale = stats.t.fit(sub_runs["log_pace"])
    # Simulate the same amount of random data with the estimated parameters
    simulated_data = stats.t.rvs(degrees_of_freedom, location, scale, size=len(sub_runs["log_pace"]))

    #print(f'{num_runs} ST fit medi: {location:.2f}, Sample std: {scale:.2f}, degrees_of_freedom: {degrees_of_freedom:.1f}, min: {sub_runs["pace"].min():.1f}, max: {sub_runs["pace"].max():.1f}')
    all_params.append({
        "num_runs": num_runs,
        "dof": degrees_of_freedom,
        "location": location,
        "scale": scale
    })

    # Fit a t-distribution to your data
    # Calculate robust estimates of the location and scale
    #location = sub_runs["log_pace"].median()
    #scale = stats.median_abs_deviation(sub_runs["log_pace"])

    # Fit a t-distribution to your data, fixing the location and scale
    #degrees_of_freedom, _, _ = stats.t.fit(sub_runs["log_pace"], floc=location, fscale=scale)
    #simulated_data_2 = stats.t.rvs(degrees_of_freedom, location, scale, size=len(sub_runs["log_pace"]))

    #print(f'{num_runs} ST fit medi: {location:.2f}, Sample std: {scale:.2f}, degrees_of_freedom: {degrees_of_freedom:.1f}, min: {sub_runs["pace"].min():.1f}, max: {sub_runs["pace"].max():.1f}')
    fig, ax = plt.subplots()
    sns.histplot(sub_runs["log_pace"], bins=30, color='blue', kde=True, label='Original Data', alpha=0.4, ax=ax)
    sns.histplot(simulated_data, bins=30, color='red', kde=True, label='Simulated Data', alpha=0.4, ax=ax)
    ax.set_title(f'num_runs = {num_runs}, n = {len(sub_runs)}, dof = {degrees_of_freedom:.1f}, std = {scale:.2f}')
    ax.legend()
    #sns.histplot(simulated_data_2, bins=30, color='green', kde=True, label='Simulated Data', alpha=0.4, ax=ax)

    

In [ ]:
pd.json_normalize(all_params)

In [ ]:
sns.displot(x="log_pace", hue="num_runs", kind="kde", height=6, aspect=1.7, palette="bright", data=runs)

In [ ]:
# Fit a t-distribution to the data in the specified column
df, loc, scale = stats.t.fit(runs["log_pace"])

# Calculate the CDF values of the fitted t-distribution for each data point
cdf_values = stats.t.cdf(runs["log_pace"], df, loc, scale)

# Perform the K-S test
test_statistic, p_value = stats.kstest(runs["log_pace"], cdf_values)

# Print the results
print(f'Test statistic: {test_statistic}')
print(f'p-value: {p_value}')